In [4]:
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"
path = "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

--2020-08-17 09:14:44--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 173.194.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2020-08-17 09:14:44 (221 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [9]:

def train_happy_sad_model():

    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('acc')>DESIRED_ACCURACY):
                print("\nReached 99.9% accuracy so cancelling training!")
                self.model.stop_training = True

    callbacks = myCallback()
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])
        


    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255)


    train_generator = train_datagen.flow_from_directory(
       '/tmp/h-or-s/', 
        target_size=(150, 150), 
        batch_size=10,
        class_mode='binary')
        
    history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=20,
      verbose=1, callbacks=[callbacks])
    
    return history.history['acc'][-1]

In [10]:
train_happy_sad_model()

Found 80 images belonging to 2 classes.
Epoch 1/20
8/8 [==============================] - 2s 243ms/step - loss: 4.3723 - acc: 0.4875
Epoch 2/20
8/8 [==============================] - 2s 245ms/step - loss: 0.7305 - acc: 0.5500
Epoch 3/20
8/8 [==============================] - 2s 244ms/step - loss: 0.5781 - acc: 0.6250
Epoch 4/20
8/8 [==============================] - 2s 242ms/step - loss: 0.4923 - acc: 0.7250
Epoch 5/20
8/8 [==============================] - 2s 242ms/step - loss: 0.2153 - acc: 0.9500
Epoch 6/20
8/8 [==============================] - 2s 243ms/step - loss: 0.2654 - acc: 0.9000
Epoch 7/20
8/8 [==============================] - 2s 241ms/step - loss: 0.0886 - acc: 0.9875
Epoch 8/20
8/8 [==============================] - 2s 241ms/step - loss: 0.0717 - acc: 0.9750
Epoch 9/20
8/8 [==============================] - 2s 243ms/step - loss: 0.1039 - acc: 0.9375
Epoch 10/20
8/8 [==============================] - 2s 242ms/step - loss: 0.0845 - acc: 0.9500
Epoch 11/20
8/8 [============

1.0